# Import Library

In [ ]:
import mne
import numpy as np
import os
import os.path as op
import matplotlib.pyplot as plt
import nibabel as nib
from mne.datasets import sample
from mne.minimum_norm import make_inverse_operator, apply_inverse_epochs
from mne.datasets import fetch_fsaverage
import scipy.io
from scipy.io import loadmat
from scipy.spatial import Delaunay
import PIL
from PIL import Image
import datetime
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import Sequential
from keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D, Dense, Flatten, Concatenate, BatchNormalization, Dropout, Input
from keras.layers.merge import concatenate
from tensorflow.keras.optimizers import Adam
# Load the TensorBoard notebook extension
%load_ext tensorboard
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold
import gc
%matplotlib inline

DIRECTORY_PATH = os.getcwd()

# Preprocess Data

In [ ]:
channels_mapping = {
    "EEG-Fz": "Fz",
    "EEG-0": "FC3",
    "EEG-1": "FC1",
    "EEG-2": "FCz",
    "EEG-3": "FC2",
    "EEG-4": "FC4",
    "EEG-5": "C5",
    "EEG-C3": "C3", 
    "EEG-6": "C1",
    "EEG-Cz": "Cz",
    "EEG-7": "C2",
    "EEG-C4": "C4",
    "EEG-8": "C6",
    "EEG-9": "CP3",
    "EEG-10": "CP1",
    "EEG-11": "CPz",
    "EEG-12": "CP2",
    "EEG-13": "CP4",
    "EEG-14": "P1",
    "EEG-Pz": "Pz",
    "EEG-15": "P2",
    "EEG-16": "POz",
    "EOG-left": "EOG-left",
    "EOG-central": "EOG-central",
    "EOG-right": "EOG-right"
}

channels_type_mapping = {
    "Fz": "eeg",
    "FC3": "eeg",
    "FC1": "eeg",
    "FCz": "eeg",
    "FC2": "eeg",
    "FC4": "eeg",
    "C5": "eeg",
    "C3": "eeg", 
    "C1": "eeg",
    "Cz": "eeg",
    "C2": "eeg",
    "C4": "eeg",
    "C6": "eeg",
    "CP3": "eeg",
    "CP1": "eeg",
    "CPz": "eeg",
    "CP2": "eeg",
    "CP4": "eeg",
    "P1": "eeg",
    "Pz": "eeg",
    "P2": "eeg",
    "POz": "eeg",
    "EOG-left": "eog",
    "EOG-central": "eog",
    "EOG-right": "eog"
}

In [ ]:
img = nib.load("/Users/ivanl/Downloads/MRIcron_windows/MRIcron/Resources/templates/brodmann.nii.gz")

brodmann_data = img.get_fdata()
brodmann_motor = brodmann_data.reshape(-1) == 4
print(brodmann_motor)

shape, affine = img.shape[:3], img.affine
coords = np.array(np.meshgrid(*(range(i) for i in shape), indexing='ij'))
coords = np.rollaxis(coords, 0, len(shape) + 1)
mm_coords = nib.affines.apply_affine(affine, coords)

def in_hull(p, hull):
    """
    Test if points in `p` are in `hull`

    `p` should be a `NxK` coordinates of `N` points in `K` dimensions
    `hull` is either a scipy.spatial.Delaunay object or the `MxK` array of the 
    coordinates of `M` points in `K`dimensions for which Delaunay triangulation
    will be computed
    """
    if not isinstance(hull,Delaunay):
        hull = Delaunay(hull)

    return hull.find_simplex(p)>=0

my_left_points = None
my_right_points = None

In [ ]:
""""
labels utility function
"""
def load_subject_labels(name="A01E.mat", dir="drive/Shareddrives/Motor Imagery/BCI competition IV dataset/2a/2a true_labels/"):
  data = scipy.io.loadmat(dir + name)["classlabel"].reshape(-1)
  return data

def load_all_true_labels(dataset_path):
  data = {}
  for root, dirs, files in os.walk(dataset_path):
    for file in files:
      data[file] = load_subject_labels(name=file, dir=root) 
  return data

"""
plot graph utility function
"""
def plot_average_graph(subject_name="A01T.gdf", Class="left", filter_channels=None):
  average = {"left": None, "right": None, "foot": None, "tongue": None, "unknown": None}
  for event_class, event_data in data[subject_name]["epoch_data"].items():
    if event_data != []:
      average[event_class] = np.transpose(np.mean(event_data, axis=0))

  x = average[Class]
  if filter_channels is None:
    fig, axs = plt.subplots(x.shape[1], gridspec_kw={'hspace': 0})
    fig.set_size_inches(37, 21)
    for channel in range(x.shape[1]):
      axs[channel].title.set_text(ch_names[channel])
      axs[channel].title.set_size(20)
      axs[channel].title.set_y(0.7)
      axs[channel].plot(range(x.shape[0]), x[:, channel])
      axs[channel].axvline(x=250, color="r", linestyle='--')
      #axs[channel].axvline(x=875, color="r", linestyle='--')
  else :
    fig, axs = plt.subplots(len(filter_channels), gridspec_kw={'hspace': 0})
    fig.set_size_inches(37, 10.5)
    for i in range(len(filter_channels)):
      for channel in range(x.shape[1]):
        if(filter_channels[i] == ch_names[channel]):
          axs[i].title.set_text(ch_names[channel])
          axs[i].title.set_size(20)
          axs[i].title.set_y(0.7)
          axs[i].plot(range(x.shape[0]), x[:, channel])
          axs[i].axvline(x=250, color="r", linestyle='--')
          #axs[i].axvline(x=875, color="r", linestyle='--')
          break
  plt.tight_layout()

def plot_multiple_graph(subject_name="A02T.gdf", classes=["left", "right", "foot", "tongue"], filter_channels=None):
  average = {"left": None, "right": None, "foot": None, "tongue": None, "unknown": None}
  for event_class, event_data in data[subject_name]["epoch_data"].items():
    if event_data != []:
      average[event_class] = np.transpose(np.mean(event_data, axis=0))

  color = {"left": "b", "right": "g", "foot": "c", "tongue": "m", "tongue": "y"}
  x = []
  for Class in classes:
    x.append(average[Class])

  if filter_channels is None:
    fig, axs = plt.subplots(x[0].shape[1], gridspec_kw={'hspace': 0})
    fig.set_size_inches(37, 21)
    for channel in range(x[0].shape[1]):
      axs[channel].title.set_text(ch_names[channel])
      axs[channel].title.set_size(20)
      axs[channel].title.set_y(0.7)
      axs[channel].axvline(x=250, color="r", linestyle='--')
      #axs[channel].axvline(x=875, color="r", linestyle='--')
      for i in range(len(classes)):
        axs[channel].plot(range(x[i].shape[0]), x[i][:, channel], color=color[classes[i]])
  else:
    fig, axs = plt.subplots(len(filter_channels), gridspec_kw={'hspace': 0})
    fig.set_size_inches(37, 10.5)
    for i in range(len(filter_channels)):
      for channel in range(x[0].shape[1]):
        if(filter_channels[i] == ch_names[channel]):
          axs[i].title.set_text(ch_names[channel])
          axs[i].title.set_size(20)
          axs[i].title.set_y(0.7)
          axs[i].axvline(x=250, color="r", linestyle='--')
          #axs[i].axvline(x=875, color="r", linestyle='--')
          for j in range(len(classes)):
            axs[i].plot(range(x[j].shape[0]), x[j][:, channel], color=color[classes[j]])
          break
  plt.tight_layout()

"""
load data function
"""
def load_subject(name="A01T.gdf", dir='drive/Shareddrives/Motor Imagery/BCI competition IV dataset/2a/BCICIV_2a_gdf/', debug=None):
  subject_data = {}
  # Load data
  raw = mne.io.read_raw_gdf(dir + name)
  # Rename channels
  raw.rename_channels(channels_mapping)
  # Set channels types
  raw.set_channel_types(channels_type_mapping)
  # Set montage
  # Read and set the EEG electrode locations
  ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')
  raw.set_montage(ten_twenty_montage)
  # Set common average reference
  raw.set_eeg_reference('average', projection=True, verbose=False)
  # Drop eog channels
  raw.drop_channels(["EOG-left", "EOG-central", "EOG-right"])

  subject_data["raw"] = raw
  subject_data["info"] = raw.info
  if debug == "all":
    print("-------------------------------------------------------------------------------------------------------")
    for key, item in raw.info.items():
      print(key, item)
    print("-------------------------------------------------------------------------------------------------------")
  
  """
  '276': 'Idling EEG (eyes open)'
  '277': 'Idling EEG (eyes closed)'
  '768': 'Start of a trial'
  '769': 'Cue onset left (class 1)'
  '770': 'Cue onset right (class 2)'
  '771': 'Cue onset foot (class 3)'
  '772': 'Cue onset tongue (class 4)'
  '783': 'Cue unknown'
  '1023': 'Rejected trial'
  '1072': 'Eye movements'
  '32766': 'Start of a new run'
  """
  custom_mapping = {'276': 276, '277': 277, '768': 768, '769': 769, '770': 770, '771': 771, '772': 772, '783': 783, '1023': 1023, '1072': 1072, '32766': 32766}
  events_from_annot, event_dict = mne.events_from_annotations(raw, event_id=custom_mapping)

  if debug == " all":
    print("-------------------------------------------------------------------------------------------------------")
    print(event_dict)
    print(events_from_annot)
    print("-------------------------------------------------------------------------------------------------------")
    
    print("-------------------------------------------------------------------------------------------------------")
    for i in range(len(raw.annotations)):
      print(events_from_annot[i], raw.annotations[i])  
    print("-------------------------------------------------------------------------------------------------------")

  class_info = "Idling EEG (eyes open): " + str(len(events_from_annot[events_from_annot[:, 2]==276][:, 0])) + "\n" + \
               "Idling EEG (eyes closed): " + str(len(events_from_annot[events_from_annot[:, 2]==277][:, 0])) + "\n" + \
               "Start of a trial: " + str(len(events_from_annot[events_from_annot[:, 2]==768][:, 0])) + "\n" + \
               "Cue onset left (class 1): " + str(len(events_from_annot[events_from_annot[:, 2]==769][:, 0])) + "\n" + \
               "Cue onset right (class 2): " + str(len(events_from_annot[events_from_annot[:, 2]==770][:, 0])) + "\n" + \
               "Cue onset foot (class 3): " + str(len(events_from_annot[events_from_annot[:, 2]==771][:, 0])) + "\n" + \
               "Cue onset tongue (class 4): " + str(len(events_from_annot[events_from_annot[:, 2]==772][:, 0])) + "\n" + \
               "Cue unknown: " + str(len(events_from_annot[events_from_annot[:, 2]==783][:, 0])) + "\n" + \
               "Rejected trial: " + str(len(events_from_annot[events_from_annot[:, 2]==1023][:, 0])) + "\n" + \
               "Eye movements: " + str(len(events_from_annot[events_from_annot[:, 2]==1072][:, 0])) + "\n" + \
               "Start of a new run: " + str(len(events_from_annot[events_from_annot[:, 2]==32766][:, 0]))
  subject_data["class_info"] = class_info

  if debug == "all" or debug == "important":
    print("-------------------------------------------------------------------------------------------------------")
    print(class_info)
    print("-------------------------------------------------------------------------------------------------------")

  epoch_data = {"left": [], "right": [], "foot": [], "tongue": [], "unknown": []}
  rejected_trial = events_from_annot[events_from_annot[:, 2]==1023][:, 0]
  class_dict = {"left": 769, "right": 770, "foot": 771, "tongue": 772, "unknown": 783}
  raw_data = raw.get_data() #(22, 672528)
  start = 0                 # cue
  stop = 500                # cue+3.0s

  for event_class, event_id in class_dict.items():
    current_event = events_from_annot[events_from_annot[:, 2]==event_id][:, 0]
    if event_class == "unknown":
      subject_true_labels = true_labels[name[:4]+".mat"]
      class_dict_labels = {1: "left", 2: "right", 3: "foot", 4: "tongue"}
      for i in range(len(current_event)):
        # exclude artifact
        if (current_event[i] - 500 != rejected_trial).all():
          current_event_data = np.expand_dims(np.array(raw_data[:22, current_event[i]+start:current_event[i]+stop]), axis=0)
          if (epoch_data.get(class_dict_labels[subject_true_labels[i]]) == None).all():
            epoch_data[class_dict_labels[subject_true_labels[i]]] = current_event_data
          else:
            epoch_data[class_dict_labels[subject_true_labels[i]]] = np.append(epoch_data[class_dict_labels[subject_true_labels[i]]], current_event_data, axis=0)
    else:
      for i in range(len(current_event)):
        # exclude artifact
        if((current_event[i] - 500 != rejected_trial).all()):
          epoch_data[event_class].append(np.array(raw_data[:22, current_event[i]+start:current_event[i]+stop]))
      epoch_data[event_class] = np.array(epoch_data[event_class])

  if debug == "all" or debug == "important":
    print("-------------------------------------------------------------------------------------------------------")
    for key, data in epoch_data.items():
      print(key, len(data))
    print("-------------------------------------------------------------------------------------------------------")

  for event_class, event_data in epoch_data.items():
    epoch_data[event_class] = np.array(event_data)

  subject_data["epoch_data"] = epoch_data
    

  return subject_data

def load_all_subject(dataset_path):
  data = {}
  for root, dirs, files in os.walk(dataset_path):
    for file in files:
      data[file] = load_subject(name=file, dir=root) 
  return data

In [ ]:
# cd to google drive
os.chdir("G:")

# Download fsaverage files
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = op.dirname(fs_dir)

# The files live in:
subject = 'fsaverage'
trans = 'fsaverage'  # MNE has a built-in fsaverage transformation
src = op.join(fs_dir, 'bem', 'fsaverage-ico-5-src.fif')
bem = op.join(fs_dir, 'bem', 'fsaverage-5120-5120-5120-bem-sol.fif')

source = mne.read_source_spaces(src)
left = source[0]
right = source[1]
left_pos = left["rr"][left["inuse"]==1]
right_pos = right["rr"][right["inuse"]==1]
                        
transformation = mne.read_trans(op.join(fs_dir, "bem", "fsaverage-trans.fif"))

save_path = op.join(os.getcwd(), "Shared drives", "Motor Imagery", "Source Estimate")

In [ ]:
true_labels_path = "Shared drives/Motor Imagery/BCI competition IV dataset/2a/2a true_labels/"
true_labels = load_all_true_labels(true_labels_path)

dataset_path = 'Shared drives/Motor Imagery/BCI competition IV dataset/2a/BCICIV_2a_gdf/'
data = load_all_subject(dataset_path)

In [ ]:
# some information to help to understand functions and data structure
"""
for key, item in data.items():
  print(key)

ch_names = data["A01T.gdf"]["info"]["ch_names"]
print(ch_names)

print(data["A01T.gdf"]["class_info"])

for key, value in data.items():
  print(key)
  for event_class, event_data in value["epoch_data"].items():
      print(event_class, len(event_data))
  print()

subject_name = "A01T.gdf"
Class = "left"
filter_channels = ["C3", "Cz", "C4"]
plot_average_graph(subject_name, Class, filter_channels)

subject_name = "A02T.gdf"
classes = ["left", "right"]
filter_channels = ["C3", "Cz", "C4"]
plot_multiple_graph(subject_name, classes, filter_channels)
"""

In [ ]:
def create_epochs(data):
  subjects_data = {}

  for subject in data.keys():
    epochs_data = {}
    for event in data[subject]["epoch_data"].keys():
      if data[subject]["epoch_data"][event].any():
        epochs_data[event] = mne.EpochsArray(data[subject]["epoch_data"][event], data[subject]["info"])
    subjects_data[subject] = epochs_data

  return subjects_data

epochs = create_epochs(data)

In [ ]:
my_epochs = epochs["A01T.gdf"]["right"]
my_evoked = my_epochs.average().pick("eeg")

noise_cov = mne.compute_covariance(my_epochs, tmax=0., method=['shrunk', 'empirical'], rank=None, verbose=False)
fwd = mne.make_forward_solution(my_epochs.info, trans=trans, src=src,
                            bem=bem, eeg=True, meg=False, mindist=5.0, n_jobs=1)
# forward matrix
fwd_fixed = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                         use_cps=True)

inverse_operator = make_inverse_operator(
    my_epochs.info, fwd, noise_cov, loose=0.2, depth=0.8)

method = "sLORETA"
snr = 3.
lambda2 = 1. / snr ** 2
stc = mne.minimum_norm.apply_inverse(my_evoked, inverse_operator, lambda2,
                              method=method, pick_ori="normal", verbose=True)

reconstruct_evoked = mne.apply_forward(fwd_fixed, stc, my_evoked.info)
my_evoked.plot_topomap()
reconstruct_evoked.plot_topomap()

# CNN Classification

In [ ]:
"""
labels
left (class 0) right (class 1) foot (class 2) tongue (class 3)

channels
c3(7) cz(9) c4(11)
"""

results = {"A01": {}, "A02": {}, "A03": {}, "A04": {}, "A05": {}, "A06": {}, "A07": {}, "A08": {}, "A09": {}}
labels = {"left": 0, "right": 1}
select_channels = [7, 9, 11]
debug = True
individual = True

# train model on each subject individually
# data_list = []
# if individual:
#   for subject in results.keys():
#     data_list.append([subject+"T.gdf", subject+"E.gdf"])
# else:
#   data_list.append(data.keys())

# train model on best two subjects
data_list = []
data_list.append(["A03T.gdf", "A03E.gdf", "A09T.gdf", "A09E.gdf"])

for data_name in data_list:
  X = None
  Y = None
  for name in data_name:
    for event_class, event_data in data[name]["epoch_data"].items():
      if event_data.size != 0 and event_class in labels:
        data_samples = None
        for select_channel in select_channels:
          data_sample = np.expand_dims(event_data[:, select_channel, :], axis=1)
          if data_samples is not None:
            data_samples = np.append(data_samples, data_sample, axis=1)
          else:
            data_samples = data_sample
        event_data = np.array(data_samples)
        if X is None:
          X = event_data
          Y = np.ones(len(event_data), dtype=int) * int(labels[event_class])
        else:
          X = np.append(X, event_data, axis=0)
          Y = np.append(Y, np.ones(len(event_data), dtype=int) * int(labels[event_class]))


  Zxx = tf.signal.stft(X, frame_length=256, frame_step=16)
  Zxx = tf.abs(Zxx)
  Zxx = Zxx.numpy() 

  if debug:
    print(data_name)
    print("shape of X and Y: " + str(X.shape) + " " + str(Y.shape))
    print("shape of Zxx: " + str(Zxx.shape))

    #samples = 0
    #print(Y[samples])
    #log_spec = np.log(Zxx[samples][0].T)
    #height = 40
    #width = log_spec.shape[1]
    #X = np.linspace(0, 2, num=width)
    #Y = range(height)
    #plt.pcolormesh(X, Y, log_spec[:40, ])
    #plt.title('STFT Magnitude')
    #plt.ylabel('Frequency [Hz]')
    #plt.xlabel('Time [sec]')
    #plt.show()

  # preprocess data
  rgb_weights = [0.2989, 0.5870, 0.1140]
  X = None

  # spectrogram
  left_mean_img = {"c3": [], "cZ": [], "c4": []}
  right_mean_img = {"c3": [], "cZ": [], "c4": []}

  # convert stft image to numpy array
  for i in range(Zxx.shape[0]):
    current_image = None
    current_data = Zxx[i][:, :, :40]

    for channel in range(current_data.shape[0]):
      fig = plt.figure(figsize=(16, 40), dpi=1)
      plot = fig.add_subplot(111)

      log_spec = np.log(current_data[channel].T)
      height = log_spec.shape[0]
      width = log_spec.shape[1]
      x_axis = np.linspace(0, 2, num=width)
      y_axis = range(height)
      plot.pcolormesh(x_axis, y_axis, log_spec)
      plot.axis('off')
      fig.tight_layout(pad=0)
      fig.canvas.draw()

      img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
      img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))

      plt.close(fig)
      img = np.array(img, dtype=np.float32) / 255

      # show rgb image
      #my_img = Image.fromarray(np.uint8(img*255))
      #display(my_img.resize((16*10, 40*10), PIL.Image.NONE))

      # convert rgb to gray scale
      img = np.dot(img[...,:3], rgb_weights)
      img = np.expand_dims(img, axis = 2)

      if Y[i] == 0:
        if channel == 0 :
          left_mean_img["c3"].append(img)
        elif channel == 1:
          left_mean_img["cZ"].append(img)
        else:
          left_mean_img["c4"].append(img)
      else:
        if channel == 0 :
          right_mean_img["c3"].append(img)
        elif channel == 1:
          right_mean_img["cZ"].append(img)
        else:
          right_mean_img["c4"].append(img)

      # show grayscale image
      #my_img = Image.fromarray(np.uint8(img[:, :, 0]*255), 'L')
      #display(my_img.resize((16*10, 40*10), PIL.Image.NONE))

      if current_image is None:
        current_image = img
      else:
        current_image = np.append(current_image, img, axis=1)

    current_image = np.expand_dims(current_image, axis=0)

    if X is None:
      X = current_image
    else:
      X = np.append(X, current_image, axis=0)

  if debug:
    print(X.shape)
    print(Y.shape)

    print("spectrogram mean")
    left_mean_img_c3 = np.mean(np.array(left_mean_img["c3"]), axis=0)
    left_mean_img_cZ = np.mean(np.array(left_mean_img["cZ"]), axis=0)
    left_mean_img_c4 = np.mean(np.array(left_mean_img["c4"]), axis=0)
    right_mean_img_c3 = np.mean(np.array(right_mean_img["c3"]), axis=0)
    right_mean_img_cZ = np.mean(np.array(right_mean_img["cZ"]), axis=0)
    right_mean_img_c4 = np.mean(np.array(right_mean_img["c4"]), axis=0)

    # show image
    """
    print("left_mean_img_c3")
    my_img = Image.fromarray(np.uint8(left_mean_img_c3[:, :, 0]*255), 'L')
    display(my_img.resize((16*10, 40*10), PIL.Image.NONE))
    print("left_mean_img_cZ")
    my_img = Image.fromarray(np.uint8(left_mean_img_cZ[:, :, 0]*255), 'L')
    display(my_img.resize((16*10, 40*10), PIL.Image.NONE))
    print("left_mean_img_c4")
    my_img = Image.fromarray(np.uint8(left_mean_img_c4[:, :, 0]*255), 'L')
    display(my_img.resize((16*10, 40*10), PIL.Image.NONE))
    print("right_mean_img_c3")
    my_img = Image.fromarray(np.uint8(right_mean_img_c3[:, :, 0]*255), 'L')
    display(my_img.resize((16*10, 40*10), PIL.Image.NONE))
    print("right_mean_img_cZ")
    my_img = Image.fromarray(np.uint8(right_mean_img_cZ[:, :, 0]*255), 'L')
    display(my_img.resize((16*10, 40*10), PIL.Image.NONE))
    print("right_mean_img_c4")
    my_img = Image.fromarray(np.uint8(right_mean_img_c4[:, :, 0]*255), 'L')
    display(my_img.resize((16*10, 40*10), PIL.Image.NONE))
    """
    print("diff_c3")
    my_img = Image.fromarray(np.uint8(((left_mean_img_c3[:, :, 0]-right_mean_img_c3[:, :, 0])+1)/2*255), 'L')
    display(my_img.resize((16*10, 40*10), PIL.Image.NONE))
    print("diff_cZ")
    my_img = Image.fromarray(np.uint8(((left_mean_img_cZ[:, :, 0]-right_mean_img_cZ[:, :, 0])+1)/2*255), 'L')
    display(my_img.resize((16*10, 40*10), PIL.Image.NONE))
    print("diff_c4")
    my_img = Image.fromarray(np.uint8(((left_mean_img_c4[:, :, 0]-right_mean_img_c4[:, :, 0])+1)/2*255), 'L')
    display(my_img.resize((16*10, 40*10), PIL.Image.NONE))

  class baseline_model(tf.keras.Model):
      def __init__(self, channels=22):
          super(baseline_model, self).__init__()
          self.conv_01 = Conv2D(filters=4, kernel_size=(3, 3), strides=(1, 1), padding='same', activation="selu")
          self.batch_norm_01 = BatchNormalization()
          self.max_pool_01 = MaxPool2D(pool_size=(2, 2), strides=(1, 1), padding="valid")
          self.conv_02 = Conv2D(filters=8, kernel_size=(3, 3), strides=(1, 1), padding='same', activation="selu")
          self.batch_norm_02 = BatchNormalization()
          self.max_pool_02 = MaxPool2D(pool_size=(2, 2), strides=(1, 1), padding="valid")
          self.flatten = Flatten()
          self.dense_01 = Dense(50, activation="selu")
          self.dense_02 = Dense(1, activation="sigmoid")

      def call(self, inputs):
          x = self.conv_01(inputs)
          x = self.batch_norm_01(x)
          x = self.max_pool_01(x)
          x = self.conv_02(x)
          x = self.batch_norm_02(x)
          x = self.max_pool_02(x)
          x = self.flatten(x)
          x = self.dense_01(x)
          x = self.dense_02(x)
          return x

  optimizer = Adam(learning_rate=1e-5)

  kfold = 10
  accuracy = 0
  precision = 0
  recall = 0
  f1 = 0

  skf = StratifiedKFold(n_splits=kfold, shuffle=True)
  skf.get_n_splits(X, Y)
  for train_index, test_index in skf.split(X, Y):
    #print(len(train_index), len(test_index))
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]

    model = baseline_model()
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
    log_dir = DIRECTORY_PATH + "/logs/" + data_name[0][:3] + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=200, callbacks=[tensorboard_callback], verbose=0)

    Y_hat = model.predict(X_test)
    Y_hat = (Y_hat >= 0.5)
    #print("accuracy: " + str(accuracy_score(Y_test, Y_hat))) 
    #print("precision:" + str(precision_score(Y_test, Y_hat, average="macro")))
    #print("recall:" + str(recall_score(Y_test, Y_hat, average="macro")))
    #print("f1:" + str(f1_score(Y_test, Y_hat, average="macro")))
    #print()  
    accuracy += accuracy_score(Y_test, Y_hat)
    precision += precision_score(Y_test, Y_hat, average="macro")
    recall += recall_score(Y_test, Y_hat, average="macro")
    f1 += f1_score(Y_test, Y_hat, average="macro")

  accuracy /= kfold
  precision /= kfold
  recall /= kfold
  f1 /= kfold
  if debug:
    print("accuracy: " + str(accuracy))
    print("precision: " + str(precision))
    print("recall: " + str(recall))
    print("f1: " + str(f1))

  results[data_name[0][:3]]["accuracy"] = accuracy
  results[data_name[0][:3]]["precision"] = precision
  results[data_name[0][:3]]["recall"] = recall
  results[data_name[0][:3]]["f1"] = f1

In [ ]:
# Calculate average performance
average_accuracy = 0
average_precision = 0
average_recall = 0
average_f1 = 0
for key, value in results.items():
  average_accuracy += value["accuracy"]
  average_precision += value["precision"]
  average_recall += value["recall"]
  average_f1 += value["f1"]

average_accuracy /= 9
average_precision /= 9
average_recall /= 9
average_f1 /= 9

print("average accuracy: " + str(average_accuracy))
print("average precision: " + str(average_precision))
print("average recall: " + str(average_recall))
print("average f1: " + str(average_f1))